In [19]:
from collections import defaultdict
from nltk.corpus import wordnet as wn
from csv import DictReader, DictWriter
from nltk.tokenize import TreebankWordTokenizer
kTOKENIZER = TreebankWordTokenizer()
import nltk
import sys, gzip, codecs
reader = codecs.getreader('utf8')
writer = codecs.getwriter('utf8')
def morphy_stem(word):
    """
    Simple stemmer
    """
    stem = wn.morphy(word)
    if stem:
        return stem.lower()
    else:
        return word.lower()

def features(text):
    d = defaultdict(int)
    for ii in kTOKENIZER.tokenize(text):
        d[morphy_stem(ii)] += 1
    return d
def prepfile(fh, code):
  if type(fh) is str:
    fh = open(fh, code)
  ret = gzip.open(fh.name, code if code.endswith("t") else code+"t") if fh.name.endswith(".gz") else fh
  if sys.version_info[0] == 2:
    if code.startswith('r'):
      ret = reader(fh)
    elif code.startswith('w'):
      ret = writer(fh)
    else:
      sys.stderr.write("I didn't understand code "+code+"\n")
      sys.exit(1)
  return ret

In [12]:
b1=features('When winter winds were wild wild.')
for k, v in b1.items():
    print k, v

be 1
winter 1
when 1
. 1
wild 2
wind 1


In [14]:
print kTOKENIZER.tokenize('When winter winds were wild wild.')
print nltk.pos_tag(kTOKENIZER.tokenize('When winter winds were wild wild.'))

['When', 'winter', 'winds', 'were', 'wild', 'wild', '.']
[('When', 'WRB'), ('winter', 'NN'), ('winds', 'NNS'), ('were', 'VBD'), ('wild', 'JJ'), ('wild', 'JJ'), ('.', '.')]


In [74]:
cnt = 0
len_dic = {}
frequent  = {}
for i in prepfile('train.tsv', 'r'):
    #print kTOKENIZER.tokenize(i.strip('\n').split('\t')[1])
    for ii in kTOKENIZER.tokenize(i.strip('\n').split('\t')[1]):
        if ii == ',':
            print ii
            cnt += 1
    if cnt>10:
        break
        #if i not in frequent:
            #frequent[i] = 1
        #else:
            #frequent[i] += 1
        #if len(i) not in len_dic:
            #len_dic[len(i)] = 1
        #else:
            #len_dic[len(i)] += 1


,
,
,
,
,
,
,
,
,
,
,


In [37]:
print len(len_dic)
for k in len_dic.keys():
    print k, len_dic[k]

 17
1 4958
2 4543
3 5415
4 6365
5 3997
6 2363
7 1506
8 927
9 431
10 195
11 66
12 36
13 24
14 12
15 4
16 2
17 1


In [52]:
ff = sorted([(k, v) for k, v in frequent.items()], key=lambda x: x[1])
print len(ff)
print ff[:10]
print ff[-10:]
print ff[4915:4916]

4916
[(u'fawn', 1), (u'Savage', 1), (u'pardon', 1), (u'ladybird', 1), (u'Redeemer', 1), (u'decays', 1), (u'Lean', 1), (u'four', 1), (u'rain-clouds', 1), (u'hanging', 1)]
[(u'my', 412), (u'and', 434), (u'And', 440), (u'to', 465), (u'I', 482), (u'of', 482), (u';', 532), (u'.', 619), (u'the', 757), (u',', 2346)]
[(u',', 2346)]


In [60]:
fd = {}
prev = 0
for i in xrange(1,11):
    print 4916*(i/float(10))
    curr = int(4916*(i/float(10)))    
    fd[i] = [x[0] for x in ff[prev : curr]]
    prev = curr

491.6
983.2
1474.8
1966.4
2458.0
2949.6
3441.2
3932.8
4424.4
4916.0


In [67]:
fdd = {}
for k, v in fd.items():
    for w in v:
        if w not in fdd:
            fdd[w] = k
        else:
            print w

In [68]:
import cPickle as pickle
pickle.dump( fdd, open( "frequent_data", "wb" ) )

In [69]:
for k, v in fdd.items():
    print k, v
    break

fawn 1


In [80]:
def ngrams(sequence, n):
    # sequence: sequence of words, list
    # n: degree of ngrams

    iter_seq = iter(sequence)
    history = []
    # make first n-1 elements ready
    while n > 1:
        history.append(next(iter_seq))
        n -= 1
    # print(history)
    # print(sequence)

    # add Nth element, yield, and get rid of first element
    # repeat the process
    for item in iter_seq:
        # print(item)
        history.append(item)
        # print(tuple(history))
        yield tuple(history)
        del history[0]

In [82]:
ngd = {}
frequent  = {}
for i in prepfile('train.tsv', 'r'):
    #print kTOKENIZER.tokenize(i.strip('\n').split('\t')[1])
    for ii in ngrams(kTOKENIZER.tokenize(i.strip('\n').split('\t')[1]), 2):
        if ii not in ngd:
            ngd[ii] = 1
        else:
            ngd[ii] += 1
        #if i not in frequent:
            #frequent[i] = 1
        #else:
            #frequent[i] += 1
        #if len(i) not in len_dic:
            #len_dic[len(i)] = 1
        #else:
            #len_dic[len(i)] += 1


In [83]:
len(ngd)

20040

In [85]:
ngf = sorted([(k, v) for k, v in ngd.items()], key=lambda x: x[1])
print len(ngf)
print ngf[:10]
print ngf[-10:]
print ngf[20039:20040]

20040
[((u'revealed', u'how'), 1), ((u'born', u'of'), 1), ((u'Forget', u','), 1), ((u'verse', u'to'), 1), ((u'earthly', u'cares'), 1), ((u'loves', u'I'), 1), ((u'the', u'well'), 1), ((u'murmured', u'with'), 1), ((u'Make', u'war'), 1), ((u'Against', u'thy'), 1)]
[((u'of', u'the'), 32), ((u'my', u'love'), 32), ((u'love', u','), 32), ((u'to', u'the'), 33), ((u'on', u'the'), 33), ((u'O', u','), 34), ((u'thee', u','), 35), ((u'me', u','), 46), ((u'in', u'the'), 54), ((u',', u'and'), 103)]
[((u',', u'and'), 103)]


In [90]:
ngff= {}
ngff_2= {}
prev = 0
for i in xrange(1,11):
    # print 20040*(i/float(10))
    curr = int(20040*(i/float(10)))    
    ngff[i] = [x[0] for x in ngf[prev : curr]]
    for ii in [x[0] for x in ngf[prev : curr]]:
        ngff_2[ii] = i
    print prev, curr
    prev = curr 

0 2004
2004 4008
4008 6012
6012 8016
8016 10020
10020 12024
12024 14028
14028 16032
16032 18036
18036 20040


In [93]:
for k, v in ngff.items():
    print k, len(v)

1 2004
2 2004
3 2004
4 2004
5 2004
6 2004
7 2004
8 2004
9 2004
10 2004


In [95]:
import cPickle as pickle
pickle.dump( ngff_2, open( "frequent_ngram", "wb" ) )
len(ngff_2)

20040